<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Camera-Calibration" data-toc-modified-id="Camera-Calibration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Camera Calibration</a></span></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pipeline</a></span><ul class="toc-item"><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#Testing-Ground-for-Pipeline" data-toc-modified-id="Testing-Ground-for-Pipeline-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Testing Ground for Pipeline</a></span><ul class="toc-item"><li><span><a href="#Undistort" data-toc-modified-id="Undistort-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Undistort</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-2.2.1.1"><span class="toc-item-num">2.2.1.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Binary-Image-Thresholded" data-toc-modified-id="Binary-Image-Thresholded-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Binary Image Thresholded</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-2.2.2.1"><span class="toc-item-num">2.2.2.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Perspective-Transform" data-toc-modified-id="Perspective-Transform-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Perspective Transform</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-2.2.3.1"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Identify-Lane-Line-Pixels" data-toc-modified-id="Identify-Lane-Line-Pixels-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Identify Lane-Line Pixels</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-2.2.4.1"><span class="toc-item-num">2.2.4.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Radius-of-Curvature-of-Lane-&amp;-Position-of-Vehicle-w.r.t.-Center" data-toc-modified-id="Radius-of-Curvature-of-Lane-&amp;-Position-of-Vehicle-w.r.t.-Center-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Radius of Curvature of Lane &amp; Position of Vehicle w.r.t. Center</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-2.2.5.1"><span class="toc-item-num">2.2.5.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Annotate-Image" data-toc-modified-id="Annotate-Image-2.2.6"><span class="toc-item-num">2.2.6&nbsp;&nbsp;</span>Annotate Image</a></span></li></ul></li><li><span><a href="#Main-Pipeline" data-toc-modified-id="Main-Pipeline-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Main Pipeline</a></span></li><li><span><a href="#Apply-pipeline-to-images" data-toc-modified-id="Apply-pipeline-to-images-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Apply pipeline to images</a></span></li></ul></li><li><span><a href="#Applying-Pipeline-to-Video" data-toc-modified-id="Applying-Pipeline-to-Video-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Applying Pipeline to Video</a></span></li></ul></div>

In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math
%matplotlib inline

# Camera Calibration

* Find camera matrix & distortion coefficients
* Demonstrate undistorting calibration image


In [ ]:
def calibration_undistort(images):
    '''
    Use images to calculate the camera calibration & image distortion matrices
    '''
    # Object points follow a grid, like (0,0,0), (1,0,0), (2,0,0)...
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpts = [] # 3d points in real world space
    imgpts = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        # Read in each image
        img = cv2.imread(fname)

        # Find the chessboard corners
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # Add object points & image points if found
        if ret == True:
            objpts.append(objp)
            imgpts.append(corners)
     
    # Create matrix using all the images (with corners found)
    # ret, mtx, dist, rvecs, tvecs
    _, mtx, dist, _, _ = cv2.calibrateCamera(objpts, imgpts, img.shape[1:], None, None)
    return (mtx, dist)

In [ ]:
# Load all the images & use them to the matrices 
images = glob.glob('camera_cal/calibration*.jpg')
mtx, dist = calibration_undistort(images)

In [ ]:
# TEST: Plot out each image undistorted 
images = glob.glob('camera_cal/calibration*.jpg')
for fname in images:
    # Undistort the image
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Plotting side-by-side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted and Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Save the calibration for later use
pickle.dump({'mtx': mtx, 'dist': dist}, 
            open('camera_cal/calibration_matrices.pickle', 'wb')
)

# Pipeline

For all images:

* Distortion correction image
* Create a thresholded binary image
    - Color transforms, gradients or other methods
* Perspective transform ("birds-eye view")
* Identify lane-line pixels and fit their positions with a polynomial
* Calculate radius of curvature of lane & position of vehicle with respect to center
* Result plotted down onto the road so lane area is identified clearly


## Helper Functions

In [ ]:
def read_image_from_file(img_filepath, convert=None):
    '''
    Returns the filename/filepath & image (numpy matrix) from given file path.
    '''
    img = cv2.imread(img_filepath)
    
    # Allow for color conversion
    if convert:
        img = cv2.cvtColor(img, convert)
        
    return (img_filepath, img)

def display_images(fnames_and_imgs, figsize=(24,8), cols=3, rows=None, cmap=None, convert=None):
    '''
    Display images given a list of images to display. If `rows` is `None`, 
    rows will be calculated given the number of columns `cols`
    '''
    
    # Getting the size to plot; enough rows to fit column
    if not rows:
        rows = math.ceil(len(fnames_and_imgs)/cols)
    elif rows*cols < len(fnames_and_imgs):
        print(f'Note: not all images will be displayed')
        print(f'{len(fnames_and_imgs)} images, but only {rows*cols} will be displayed')
    f, axs = plt.subplots(rows, cols, figsize=figsize)
    
    
    # Iterate over each axis to display the image
    for ax, fname_imgs_tuple in zip(axs.flat, fnames_and_imgs):
        fname, img = fname_imgs_tuple
        # Allow for color conversion
        if convert:
            img = cv2.cvtColor(img, convert)
        ax.set_title(fname)
        ax.imshow(img, cmap=cmap)
    

## Testing Ground for Pipeline

In [ ]:
# Load camera calibration matrices from earlier
cameraCalibration = pickle.load( open('camera_cal/calibration_matrices.pickle', 'rb' ) )
mtx, dist = map(cameraCalibration.get, ('mtx', 'dist'))

In [ ]:
# Read in files to convert
test_image_fnames = glob.glob('test_images/advance/*.jpg')
test_images = [read_image_from_file(img, convert=cv2.COLOR_BGR2RGB) 
                   for img in test_image_fnames]

# Display the unmodified images
display_images(test_images, cols=4, figsize=(24,8))

### Undistort

In [ ]:
# Distortion correction
def undistort_image(img, mtx, dist):
    '''
    Undistort a given image using `mtx` & `dist` matrices.
    '''
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    return undist_img

#### Test

In [ ]:
# Test undistortion
undist_imgs = [(fn,undistort_image(i,mtx,dist)) for fn,i in test_images]
display_images(undist_imgs, cols=4)

### Binary Image Thresholded

In [ ]:
def abs_sobel_thresh(img, orient='x', ksize=3, thresholds=(0,255)):
    '''
    Using OpenCV Sobel() function, get the absolute value of the x or y 
    gradient to produce a binary image based on the kernel size and the
    given thresholds.
    '''
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    
    # Take the absolute value of x or y gradient using OpenCV Sobel() function 
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize))
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize))
    
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a binary image of 1s where threshold is met, otherwise 0s
    binary_out = np.zeros_like(scaled_sobel)
    thresh_min, thresh_max = thresholds
    binary_out[(scaled_sobel > thresh_min) & (scaled_sobel < thresh_max)] = 1

    # Return the binary image
    return binary_out

def mag_thresh(img, sobel_ksize=3, thresholds=(0,255)):
    '''
    Using OpenCV Sobel() function, get the gradient magnitude of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.
    '''
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_ksize)
    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    # Create a binary image of 1s where threshold is met, otherwise 0s
    binary_out = np.zeros_like(gradmag)
    thresh_min, thresh_max = thresholds
    binary_out[(gradmag > thresh_min) & (gradmag < thresh_max)] = 1

    # Return the binary image
    return binary_out

def dir_threshold(img, sobel_ksize=3, thresholds=(0,np.pi/2)):
    '''
    Using OpenCV Sobel() function, get the gradient direction of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.    
    '''
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_ksize)
    
    # Calculate absolute value of the gradient direction, 
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    
    # Create a binary image of 1s where threshold is met, otherwise 0s 
    binary_out =  np.zeros_like(absgraddir)
    thresh_min, thresh_max = thresholds
    binary_out[(absgraddir > thresh_min) & (absgraddir < thresh_max)] = 1

    # Return the binary image
    return binary_out

In [ ]:
def full_binary_image(img, ksize=3, x_thres=(30,100), y_thres=(30,100), 
                      mag_thres=(30,100), dir_thres=(0,np.pi/2)):
    '''
    Using OpenCV Sobel() function, get the gradient direction of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.   
    '''
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(img, orient='x', ksize=ksize, thresholds=x_thres)
    grady = abs_sobel_thresh(img, orient='y', ksize=ksize, thresholds=y_thres)
    mag_binary = mag_thresh(img, sobel_ksize=ksize, thresholds=mag_thres)
    dir_binary = dir_threshold(img, sobel_ksize=ksize, thresholds=dir_thres)

    # Create a binary image of 1s where criteria is met, otherwise 0s 
    combined = np.zeros_like(dir_binary)
    # Gadients in x or y are met or the magnitue & direction are met
    combined[
        ((gradx == 1) & (grady == 1))# |  
        #((mag_binary == 1) & (dir_binary == 1))
    ] = 1
    
    return combined

#### Test

In [ ]:
# Test out the performance of the full binary threshold
binary_imgs = [
    (
        fn,
        full_binary_image(i,
                          ksize=11,
                          x_thres=(10,160), 
                          y_thres=(10,160),
                          mag_thres=(40,100), # (30,100) might be better
                          dir_thres=(np.pi/4, np.pi*2/5)
        )
    ) 
        for fn,i in test_images
]

display_images(binary_imgs, cols=4, cmap='gray')

### Perspective Transform

In [ ]:
def get_dst(img_size, offset):
    '''
    Return dst matrix by providing the image size `img_size` (2D) and the
    `offset`.
    '''
    # Getting the image destination
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    dst = np.float32([
        [offset, img_size[1]],
        [offset, 0],
        [img_size[0]-offset, 0],
        [img_size[0]-offset, img_size[1]]
    ])
    return dst
    
    
def get_src_from_lines(lines):
    '''
    Return src matrix by providing the lines to transform the image with.
    '''    
    # Source from the lane guide lines
    src = np.zeros(shape=(len(lines),2), dtype='float32')
    for i,line_pts in enumerate(lines):
        src[i,:] = line_pts
    return src
        
def get_warp_matrix(src, dst):
    '''
    Return the warp matrix for a perspective transform by providing the source
    and destination matrix
    '''
    return cv2.getPerspectiveTransform(src, dst)

def warp_image(img, M):
    '''
    Return a warped perspective image by a provided perspective warp matrix M.
    '''
    # Shape is 2D since we don't care about color
    img_size = img.shape[1], img.shape[0]
    return cv2.warpPerspective(img, M, (img_size))

In [ ]:
def warp_from_lane_guides(img, lines, offset=200, 
                          line_color=(0,255,0), show=False):
    '''
    Return a perspective transform image based on the provided quadrilateral
    representing lane lines in the original image.
    
    Note lines must go like [Bottom-Left, Top-Left, Top-Right, Bottom-Right].
    '''    
    copy = img.copy()

    line_thickness = 2
    # Create a line on image for each pair of points(that defines a line)
    for pt0,pt1 in zip(lines, lines[1:]):
        cv2.line(copy, pt0, pt1, line_color, line_thickness)

    # Display the lane guides with original image
    if show:
        fig, ax = plt.subplots(figsize=(40, 20))
        ax.imshow(copy)
    
    # Getting image and the relevant measurements
    img_size = (img.shape[1], img.shape[0])
    
    # Getting the image destination & source (via lane lines)
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    dst = get_dst(img_size, offset)
    src = get_src_from_lines(lines)
 
    # Unwarp the image
    M = get_warp_matrix(src, dst)
    Minv = get_warp_matrix(dst, src)
    warped = warp_image(img, M)

    if show:
        fig, ax = plt.subplots(figsize=(40, 20))
        ax.imshow(warped)
    
    return warped, Minv

#### Test

In [ ]:
# Geetting a straight lines test (straight_lines1.jpg)
undist = undist_imgs[5][1]

In [ ]:
# Create guides for the typical lanes needed
y_bottom = 720
y_top = 455

# Bottom-Left, Top-Left, Top-Right, Bottom-Right
lines = [
    (190, y_bottom), # bottom left
    (585, y_top), # top left
    (705, y_top), # top right
    (1130, y_bottom) # bottom right
]

In [ ]:
warped, _ = warp_from_lane_guides(undist, lines, show=True);

### Identify Lane-Line Pixels

In [ ]:
def find_lane_pixels(img, nwindows=9, margin=100, minpix=50):
    '''
    Read in an image and find the lane line locations with sliding window.
    
    Parameters
    ----------
    img : ndarray
        Numpy array representing warped image to find lane lines
    nwindows : int, optional
        Number of sliding windows (default is 9)
    margin : int, optional
        Width of the windows +/- margin (default is 100)
    minpix : int, optional
        Minimum number of pixels found to recenter window (default is 50)

    Returns
    -------
    tuple
        Tuple of leftx, lefty, rightx, righty, out_img
    
    '''
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((img, img, img))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(img):
    '''
    Read in an image and fits a polynomial for each lane line.
    
    Parameters
    ----------
    img : ndarray
        Numpy array representing warped image to find lane lines
    

    Returns
    -------
    tuple
        Tuple of out_img, left_fit, right_fit
    '''
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0] # Blue
    out_img[righty, rightx] = [0, 0, 255] # Red

    # Give out the image produced and the lane lines to plot
    return out_img, left_fit, right_fit

def get_2nd_order_poly_points(img, left_fit, right_fit):
    '''
    Produce the polynomial points for each lane to plot.
    
    Parameters
    ----------
    img : ndarray
        Numpy array representing warped image to find lane lines
    left_fit : ndarray
        Numpy array representing polynomial for left lane line
    right_fit : ndarray
        Numpy array representing polynomial for right lane line
    
    
    Returns
    -------
    tuple 
        Tuple of ploty, left_fitx, right_fitx
    '''
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    return ploty, left_fitx, right_fitx

#### Test

In [ ]:
# Perform a test for images
for i in range(8):
    fig = plt.figure(figsize=(40,60))
    ## Undistort image
    img_undist = undist_imgs[i][1]

    ## Create binary thresholds
    img_binary = full_binary_image(
                                img=img_undist,
                                ksize=5,
                                x_thres=(10,160), 
                                y_thres=(10,160),
                                mag_thres=(40,100),
                                dir_thres=(np.pi/4, np.pi*2/5)
    )

    ## Perspective transform
    # Create guides for the typical lanes needed
    y_bottom = 720
    y_top = 455
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    lane_lines = [
        (190, y_bottom), # bottom left
        (585, y_top), # top left
        (705, y_top), # top right
        (1130, y_bottom) # bottom right
    ]

    img_warped, Minv = warp_from_lane_guides(img_binary, lane_lines)

    # Getting lane lines
    img_lane_lines, left_fit, right_fit = fit_polynomial(img_warped)
    ploty, left_fitx, right_fitx = get_2nd_order_poly_points(img_warped, left_fit, right_fit)

    # Plot the undistorted and lane line mages
    plt.title(undist_imgs[i][0])
    plt.subplot(2,2,1)
    plt.imshow(img_undist)
    plt.subplot(2,2,2)
    plt.imshow(img_lane_lines)
    plt.plot(left_fitx, ploty, '--', color='yellow', linewidth=8)
    plt.plot(right_fitx, ploty, '--', color='yellow', linewidth=8)
    plt.show()


### Radius of Curvature of Lane & Position of Vehicle w.r.t. Center

In [ ]:
def measure_curvature_pixels(img, left_fit, right_fit):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def measure_curvature_real(img, left_fit_cr, right_fit_cr):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def measure_vehicle_from_center(img, left_fit_cr, right_fit_cr):
    '''
    Calculates vehicle position from center of the lanes based on found
    curvature (in meters). Note a negative value is left of the center, and a
    positive value is right of the center.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Calculate vehicle center
    xMax = img.shape[1]
    yMax = img.shape[0]
    vehicle_center = xMax / 2
    # Use previous results to find start of lanes along x-axis
    line_left = (left_fit_cr[0]*yMax)**2 + left_fit_cr[1]*yMax + left_fit_cr[2]
    line_right = (right_fit_cr[0]*yMax)**2 + right_fit_cr[1]*yMax + right_fit_cr[2]
    # Find pixel position for center of lanes.
    line_middle = (line_right - line_left)/2 + line_left

    # Convert to distance (from pixels)
    vehicle_dist_from_center = vehicle_center - line_middle
    return vehicle_dist_from_center * xm_per_pix

#### Test

In [ ]:
i = 0
for i in range(8):
    fig = plt.figure(figsize=(10,20))
    ## Undistort image
    img_undist = undist_imgs[i][1]

    ## Create binary thresholds
    img_binary = full_binary_image(
                                img=img_undist,
                                ksize=5,
                                x_thres=(10,160), 
                                y_thres=(10,160),
                                mag_thres=(40,100),
                                dir_thres=(np.pi/4, np.pi*2/5)
    )

    ## Perspective transform
    # Create guides for the typical lanes needed
    y_bottom = 720
    y_top = 455

    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    lane_lines = [
        (190, y_bottom), # bottom left
        (585, y_top), # top left
        (705, y_top), # top right
        (1130, y_bottom) # bottom right
    ]

    img_warped, Minv = warp_from_lane_guides(img_binary, lane_lines)

    # Getting lane lines  
    img_lane_lines, left_fit, right_fit = fit_polynomial(img_warped)

    # Calculate the radius of curvature in pixels for both lane lines
    left_curverad, right_curverad = measure_curvature_pixels(img_warped, left_fit, right_fit)

    print(left_curverad, right_curverad)

    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad = measure_curvature_real(img_warped,left_fit,right_fit)

    print(left_curverad, 'm', right_curverad, 'm')
    # 
    print('Distance right from the of center of the lane',
          measure_vehicle_from_center(img_warped,left_fit,right_fit),
          'm'
    )

    plt.imshow(img_undist)
    plt.show()

### Annotate Image

In [ ]:
def annotate_img(orig_img, warped_img, Minv, left_fitx, right_fitx, ploty):
    '''
    Return an annotated original image with lanes identified.
    '''
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped_img.shape[1], warped_img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(orig_img, 1, newwarp, 0.3, 0)
    
    # Add curvature information
    img_lane_lines, left_fit, right_fit = fit_polynomial(warped_img)
    left_curverad, right_curverad = measure_curvature_pixels(warped_img, left_fit, right_fit)
    dist_from_center = measure_vehicle_from_center(warped_img, left_fit, right_fit)
    # Message shows either left or right of center
    curvature_message = (f'{dist_from_center:.2f}m right of center'
                                if dist_from_center > 0
                                else f'{-dist_from_center:.2f}m left of center'
                        )

    # Draw info
    font = cv2.FONT_HERSHEY_DUPLEX
    fontColor = (255, 255, 255)
    cv2.putText(result, 'Left curvature: {:.0f} m'.format(left_curverad), (30, 50), font, 1, fontColor, 2)
    cv2.putText(result, 'Right curvature: {:.0f} m'.format(right_curverad), (30, 80), font, 1, fontColor, 2)
    cv2.putText(result, 'Vehicle is {} of center'.format(curvature_message), (30, 110), font, 1, fontColor, 2)
    
    return result

In [ ]:
# Perform a test for images
for i in range(8):
    fig = plt.figure(figsize=(40,60))
    ## Undistort image
    img_undist = undist_imgs[i][1]

    ## Create binary thresholds
    img_binary = full_binary_image(
                                img=img_undist,
                                ksize=5,
                                x_thres=(10,160), 
                                y_thres=(10,160),
                                mag_thres=(40,100),
                                dir_thres=(np.pi/4, np.pi*2/5)
    )

    ## Perspective transform
    # Create guides for the typical lanes needed
    y_bottom = 720
    y_top = 455
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    lane_lines = [
        (190, y_bottom), # bottom left
        (585, y_top), # top left
        (705, y_top), # top right
        (1130, y_bottom) # bottom right
    ]

    img_warped, Minv = warp_from_lane_guides(img_binary, lane_lines)

    # Getting lane lines
    img_lane_lines, left_fit, right_fit = fit_polynomial(img_warped)
    ploty, left_fitx, right_fitx = get_2nd_order_poly_points(img_warped, left_fit, right_fit)

    result = annotate_img(img_undist, img_warped, Minv, left_fitx, right_fitx, ploty)

    plt.imshow(result)
    plt.show()

## Main Pipeline

In [ ]:
def lane_line_pipeline(img, mtx, dist):
    '''
    '''
    ## Undistort image
    img_undist = undistort_image(img, mtx, dist)
    
    ## Create binary thresholds
    img_binary = full_binary_image(
                                img=img_undist,
                                ksize=5,
                                x_thres=(10,160), 
                                y_thres=(10,160),
                                mag_thres=(40,100),
                                dir_thres=(np.pi/4, np.pi*2/5)
    )
    
    ## Perspective transform
    # Create guides for the typical lanes needed
    y_bottom = img_undist.shape[0]
    y_top = int(455/720 * img_undist.shape[0])
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    lane_lines = [
        (int(190/1280*img_undist.shape[1]), y_bottom), # bottom left
        (int(585/1280*img_undist.shape[1]), y_top), # top left
        (int(705/1280*img_undist.shape[1]), y_top), # top right
        (int(1130/1280*img_undist.shape[1]), y_bottom) # bottom right
    ]
    # TODO: Don't re-calculate the warp matrix each time
    img_warped, Minv = warp_from_lane_guides(img_binary, lane_lines)
    
    ## Identify lane-line pixels
    # Draw lane lines onto image
    img_lane_lines, left_fit, right_fit = fit_polynomial(img_warped)
    ploty, left_fitx, right_fitx = get_2nd_order_poly_points(img_warped, left_fit, right_fit)
    
    ## Final image transformation (w/radius of curvature & position)
    img_final = annotate_img(img_undist, img_warped, Minv, left_fitx, right_fitx, ploty)
    
    return img_final

## Apply pipeline to images

In [ ]:
# Read in files to convert
test_image_fnames = glob.glob('test_images/advance/*.jpg')
test_images = [read_image_from_file(img, convert=cv2.COLOR_BGR2RGB) 
                   for img in test_image_fnames]

In [ ]:
# Load camera calibration matrices from earlier
cameraCalibration = pickle.load(open('camera_cal/calibration_matrices.pickle', 'rb'))
mtx, dist = map(cameraCalibration.get, ('mtx', 'dist'))

In [ ]:
images = [(img_fn,lane_line_pipeline(img, mtx, dist)) for img_fn,img in test_images]

display_images(images, cols=4, figsize=(24,8))

# Applying Pipeline to Video

In [ ]:
from moviepy.editor import VideoFileClip
    
def create_video(input_video_fname, output_video_fname='out.mp4'):
    '''
    Creates an annotated video by processing each frame of original video.
    ''' 
    myclip = VideoFileClip(input_video_fname)

    clip = myclip.fl_image(lambda img: lane_line_pipeline(img,mtx,dist))
    clip.write_videofile(output_video_fname, audio=False)   

In [ ]:
videos = glob.glob('test_videos/basic/challenge.mp4')
for v_name in videos:
    out_name = v_name.split('/')[-1]
    create_video(v_name, f'out_{out_name}')